In [51]:
# Loading the dataset
import datasets
dataset = datasets.load_dataset("stas/wmt16-en-ro-pre-processed", cache_dir="./wmt16-en_ro")

Found cached dataset wmt16-en-ro-pre-processed (/home/agupt135/./wmt16-en_ro/stas___wmt16-en-ro-pre-processed/enro/1.1.0/c4093132d2665734cbb5098992e5cdf3cdbd807b80a5913a456ab7cb8c34ab2b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
# imports
import os
import datetime
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa

# from official.nlp import optimization
import datasets

from tokenizers import normalizers
from tokenizers import pre_tokenizers, Tokenizer
from tokenizers import models, trainers
from transformers import pipeline
import torch

from datasets import load_dataset
from datasets import DownloadMode
from promptsource.templates import DatasetTemplates

2023-03-24 13:40:42.464297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 13:40:45.806340: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-24 13:40:45.806470: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-24 13:40:45.806484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [52]:
data = datasets.load_dataset("wmt16", "de-en", split='train')

Found cached dataset wmt16 (/home/agupt135/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)


In [55]:
# Pre process the dataset
def flatten(batch):
    batch['en'] = batch['translation']['en']
    batch['ro'] = batch['translation']['en']
    
    return batch

# Map the 'flatten' function
train = dataset['train'].map( flatten )
test = dataset['test'].map( flatten )
validation = dataset['validation'].map( flatten )

# Save to disk
train.save_to_disk("./dataset/train")
test.save_to_disk("./dataset/test")
validation.save_to_disk("./dataset/validation")

Loading cached processed dataset at /home/agupt135/wmt16-en_ro/stas___wmt16-en-ro-pre-processed/enro/1.1.0/c4093132d2665734cbb5098992e5cdf3cdbd807b80a5913a456ab7cb8c34ab2b/cache-da23343519178f9d.arrow
Loading cached processed dataset at /home/agupt135/wmt16-en_ro/stas___wmt16-en-ro-pre-processed/enro/1.1.0/c4093132d2665734cbb5098992e5cdf3cdbd807b80a5913a456ab7cb8c34ab2b/cache-4b0c9708b2ba9dcc.arrow
Loading cached processed dataset at /home/agupt135/wmt16-en_ro/stas___wmt16-en-ro-pre-processed/enro/1.1.0/c4093132d2665734cbb5098992e5cdf3cdbd807b80a5913a456ab7cb8c34ab2b/cache-6b13bf88f31cd693.arrow


Saving the dataset (0/1 shards):   0%|          | 0/610320 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1999 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1999 [00:00<?, ? examples/s]

In [50]:
train['en']

['Membership of Parliament: see Minutes',
 'Approval of Minutes of previous sitting: see Minutes',
 'Membership of Parliament: see Minutes',
 'Verification of credentials: see Minutes',
 'Documents received: see Minutes',
 'Written statements and oral questions (tabling): see Minutes',
 'Petitions: see Minutes',
 'Texts of agreements forwarded by the Council: see Minutes',
 "Action taken on Parliament's resolutions: see Minutes",
 'Agenda for next sitting: see Minutes',
 'Closure of sitting',
 '(The sitting was closed at 7.45 p.m.)',
 'Election of Vice-Presidents of the European Parliament (deadline for submitting nominations): see Minutes',
 '(The sitting was suspended at 12.40 p.m. and resumed at 3.00 p.m.)',
 'Election of Quaestors of the European Parliament (deadline for submitting nominations): see Minutes',
 '(The sitting was suspended at 3.25 p.m. and resumed at 6.00 p.m.)',
 'Agenda for next sitting: see Minutes',
 'Closure of sitting',
 '(The sitting was closed at 6.15 p.m.)',

In [3]:
# 
from tokenizers import normalizers, pre_tokenizers, Tokenizer, models, trainers

# Build a tokenizer
bpe_tokenizer = Tokenizer(models.BPE())
bpe_tokenizer.normalizer = normalizers.Lowercase()
bpe_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

trainer = trainers.BpeTrainer(
    vocab_size=50265,
    special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"],
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet(),
)

In [4]:
def batch_iterator():
    batch_length = 1000
    for i in range(0, len(train), batch_length):
        yield train[i : i + batch_length]["ro"]
        
bpe_tokenizer.train_from_iterator( batch_iterator(), length=len(train), trainer=trainer )

bpe_tokenizer.save("./ro_tokenizer.json")

# To read the tokenizer later:
# from transformers import PreTrainedTokenizerFast
# tmp = PreTrainedTokenizerFast.from_pretrained('./ro_tokenizer.json')

In [5]:
from transformers import AutoTokenizer, PreTrainedTokenizerFast

en_tokenizer = AutoTokenizer.from_pretrained( "facebook/bart-base" );
ro_tokenizer = PreTrainedTokenizerFast.from_pretrained( "./ro_tokenizer.json" );
ro_tokenizer.pad_token = en_tokenizer.pad_token

def tokenize_dataset(sample):
    input = en_tokenizer(sample['en'], padding='max_length', max_length=120, truncation=True)
    label = ro_tokenizer(sample['ro'], padding='max_length', max_length=120, truncation=True)

    input["decoder_input_ids"] = label["input_ids"]
    input["decoder_attention_mask"] = label["attention_mask"]
    input["labels"] = label["input_ids"]

    return input

train_tokenized = train.map(tokenize_dataset, batched=True)
test_tokenized = test.map(tokenize_dataset, batched=True)
validation_tokenized = validation.map(tokenize_dataset, batched=True)

/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1706: FutureWarning: Calling PreTrainedTokenizerFast.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
Loading cached processed dataset at /home/agupt135/wmt16-en_ro/stas___wmt16-en-ro-pre-processed/enro/1.1.0/c4093132d2665734cbb5098992e5cdf3cdbd807b80a5913a456ab7cb8c34ab2b/cache-6282551bc1612b35.arrow


Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Loading cached processed dataset at /home/agupt135/wmt16-en_ro/stas___wmt16-en-ro-pre-processed/enro/1.1.0/c4093132d2665734cbb5098992e5cdf3cdbd807b80a5913a456ab7cb8c34ab2b/cache-f8039ba555431d8f.arrow


In [6]:
from transformers import BartForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

model = BartForConditionalGeneration.from_pretrained(  "facebook/bart-base" )

training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    logging_steps=2,  # set to 1000 for full training
    save_steps=64,  # set to 500 for full training
    eval_steps=64,  # set to 8000 for full training
    warmup_steps=1,  # set to 2000 for full training
    max_steps=128, # delete for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True, # True if GPU
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=validation_tokenized,
)

trainer.train()

2023-03-24 11:06:27.054611: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-24 11:06:27.055184: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-24 11:06:27.055206: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: ankitgupta-nitp. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss,Validation Loss
64,2.468100,1.813501
128,1.494800,0.552713


TrainOutput(global_step=128, training_loss=4.117849241942167, metrics={'train_runtime': 106.5611, 'train_samples_per_second': 2.402, 'train_steps_per_second': 1.201, 'total_flos': 18292093747200.0, 'train_loss': 4.117849241942167, 'epoch': 0.0})

In [7]:
CUDA_LAUNCH_BLOCKING=1